In [16]:
import json
import evaluate
from tqdm import tqdm

In [2]:
rouge = evaluate.load("rouge")

In [21]:
corpus = [ json.loads(line) for line in open("../results/ppo_model/galactica-6.7b-ct2/test_with_citations.jsonl") ]

In [22]:
corpus = [ item for item in corpus if len(item["keywords"]) == 1 and len(item["citation"].split() )>=20 ]
len(corpus)

484

In [23]:
corpus_subset = []
for example in tqdm(corpus):
    score1 = rouge.compute(predictions=[ example["generated_citations"][0]["generation"]["citation"] ], references=[ example["citation"] ])["rougeL"]
    score2 = rouge.compute(predictions=[ example["generated_citations"][1]["generation"]["citation"] ], references=[ example["citation"] ])["rougeL"]
    score3 = rouge.compute(predictions=[ example["generated_citations"][2]["generation"]["citation"] ], references=[ example["citation"] ])["rougeL"]
    if score1< score2 and score2 < score3:
        example["scores"] = [score1, score2, score3]
        corpus_subset.append(example)

100%|██████████████████████████████████████████████████████████████████████████████████████| 484/484 [03:01<00:00,  2.67it/s]


In [31]:
for count, example in enumerate( corpus_subset ):    
    print(count, example["scores"])

0 [0.0909090909090909, 0.09876543209876544, 0.21052631578947364]
1 [0.16216216216216214, 0.17142857142857143, 0.23809523809523808]
2 [0.12244897959183673, 0.35714285714285715, 0.409090909090909]
3 [0.11320754716981131, 0.14285714285714285, 0.1923076923076923]
4 [0.20689655172413793, 0.4444444444444444, 0.5238095238095238]
5 [0.14634146341463414, 0.15789473684210525, 0.1951219512195122]
6 [0.17391304347826086, 0.1818181818181818, 0.2553191489361702]
7 [0.09523809523809523, 0.16326530612244897, 0.34615384615384615]
8 [0.11538461538461538, 0.1509433962264151, 0.2592592592592593]
9 [0.04081632653061224, 0.14285714285714285, 0.21739130434782608]
10 [0.10714285714285714, 0.20408163265306123, 0.2916666666666667]
11 [0.23529411764705882, 0.24, 0.2916666666666667]
12 [0.1951219512195122, 0.20512820512820512, 0.4347826086956522]
13 [0.09230769230769231, 0.14814814814814814, 0.23076923076923075]
14 [0.16666666666666669, 0.1923076923076923, 0.20408163265306123]
15 [0.13793103448275862, 0.184615384

In [114]:
example = corpus_subset[43]

In [115]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [116]:
print( color.PURPLE + color.BOLD +  "Cited Paper:" + color.END )
# print()
print( color.BOLD +  "    Title:     " + color.END + example["cited_paper_content"]["title"] )
print( color.BOLD +  "    Abstract:  " + color.END + example["cited_paper_content"]["abstract"] )
print()
print( color.PURPLE + color.BOLD +  "Manuscript:" + color.END )
# print()
print( color.BOLD +  "    Title:     " + color.END + example["citing_paper_content"]["title"] )
print( color.BOLD +  "    Abstract:  " + color.END + example["citing_paper_content"]["abstract"] )
# print()
print( color.BOLD +  "    Text Before Citation:  " + color.END + " ".join(example["text_before_citation"]) )
print()
print( color.PURPLE + color.BOLD +  "Ground-truth Citation Sentence:" + color.END )
print( "    "+example["citation"] )
print()
print( color.PURPLE + color.BOLD +  "Generated Citation Sentences:" + color.END )
# print()
print( color.DARKCYAN + color.BOLD +  "Uncontrolled Mode:" + color.END )
print( color.BOLD +  "    Citation Intent: "+ color.END +"N/A"  )
print( color.BOLD +  "    Keywords: "+ color.END +"N/A"  )
print( color.BOLD +  "    Generation: "+ color.END +example["generated_citations"][0]["generation"]["citation"]  )
score = rouge.compute( predictions=[example["generated_citations"][0]["generation"]["citation"]],
                       references=[example["citation"]]
                     )
print( color.BOLD +  "    ROUGE F1: "+ color.END + "R1: %.2f R2: %.2f RL: %.2f" %( score["rouge1"]*100,score["rouge2"]*100,score["rougeL"]*100 )  )
print()

print( color.DARKCYAN + color.BOLD +  "Intent-controlled Mode:" + color.END )
print( color.BOLD +  "    Citation Intent: "+ color.END + example["citation_intent"]  )
print( color.BOLD +  "    Keywords: "+ color.END +"N/A"  )
print( color.BOLD +  "    Generation: "+ color.END +example["generated_citations"][1]["generation"]["citation"]  )
score = rouge.compute( predictions=[example["generated_citations"][1]["generation"]["citation"]],
                       references=[example["citation"]]
                     )
print( color.BOLD +  "    ROUGE F1: "+ color.END + "R1: %.2f R2: %.2f RL: %.2f" %( score["rouge1"]*100,score["rouge2"]*100,score["rougeL"]*100 )  )
print()

print( color.DARKCYAN + color.BOLD +  "Intent- and keywords-controlled Mode:" + color.END )
print( color.BOLD +  "    Citation Intent: "+ color.END + example["citation_intent"]  )
print( color.BOLD +  "    Keywords: "+ color.END +"; ".join(example["keywords"])  )
print( color.BOLD +  "    Generation: "+ color.END +example["generated_citations"][2]["generation"]["citation"]  )
score = rouge.compute( predictions=[example["generated_citations"][2]["generation"]["citation"]],
                       references=[example["citation"]]
                     )
print( color.BOLD +  "    ROUGE F1: "+ color.END + "R1: %.2f R2: %.2f RL: %.2f" %( score["rouge1"]*100,score["rouge2"]*100,score["rougeL"]*100 )  )
print()

Cited Paper:
    Title:     Optimal Scheduling For Discounted Age Penalty Minimization In Multi-Loop Networked Control
    Abstract:  Age-of-information (AoI) is a metric quantifying information freshness at the receiver. Since AoI combines packet generation frequency, packet loss, and delay into a single metric, it has received a lot of research attention as an interface between communication network and application. In this work, we apply AoI to the problem of wireless scheduling for multi-loop networked control systems (NCS), i.e., feedback control loops closed over a shared wireless network. We model the scheduling problem as a Markov decision process (MDP) with AoI as its observable states and derive a relation of control system error and AoI. We further derive a stationary scheduling policy to minimize control error over an infinite horizon. We show that our scheduler outperforms the state-of-the-art scheduling policies for NCS. To the best of our knowledge, this is the first wor